# Content Based Recommendation

In [143]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [144]:
# Load data
data =pd.read_pickle("food.pkl")
data.head()

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,submitted_month,submitted_year,dairy-free,gluten-free,low-carb,vegan,vegetarian,recipe_id,average_rating,votes
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",...,Sep,2005,0,0,0,0,1,137739,5.000000,3
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",...,Jun,2002,0,0,0,0,0,31490,4.666667,3
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",...,Feb,2005,0,0,0,0,0,112140,4.000000,1
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",...,Apr,2003,0,0,0,0,0,59389,4.500000,2
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",...,Oct,2002,0,0,0,0,1,44061,5.000000,1


In [145]:
# Sample 10% of the dataset
sampled_data = data.sample(frac=0.1, random_state=42)

# View sample
display(sampled_data.head())

# Print the shape of the sampled dataset
print("Shape of Sampled Dataset:", sampled_data.shape)

,name,id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,...,submitted_month,submitted_year,dairy-free,gluten-free,low-carb,vegan,vegetarian,recipe_id,average_rating,votes
168477,sesame dipping sauce,353233,10,406741,2009-01-31,"['lactose', '15-minutes-or-less', 'time-to-mak...",2,"['in a small bowl , whisk together soy sauce ,...",this works well for dipping pot stickers in as...,"['soy sauce', 'rice vinegar', 'sesame oil', 'a...",...,Jan,2009,0,0,0,1,1,353233,5.0,2
201337,unattended rib roast,97973,112,152393,2004-08-17,"['time-to-make', 'main-ingredient', 'preparati...",6,"['at noon , preheat oven to 375 degrees', 'sea...",i found this recipe in the paper and thought i...,"['standing rib roast', 'salt and pepper']",...,Aug,2004,0,0,1,0,0,97973,5.0,1
173114,slow cooker mashed potatoes,265017,135,290107,2007-11-12,"['time-to-make', 'course', 'main-ingredient', ...",6,"['in a mixing bowl , combine cream cheese , so...",this recipe is from taste of home magazine. it...,"['cream cheese', 'sour cream', 'butter', 'drie...",...,Nov,2007,0,0,0,0,0,265017,5.0,2
29931,butterscotch toffee cookies,288112,25,668077,2008-02-23,"['30-minutes-or-less', 'time-to-make', 'course...",8,['mix shortening and both sugars with electric...,"in preparation for a bake sale, i changed a re...","['shortening', 'sugar', 'brown sugar', 'egg', ...",...,Feb,2008,0,0,0,0,0,288112,4.0,1
9300,au gratin hash browns casserole,94740,70,126418,2004-06-30,"['time-to-make', 'course', 'preparation', 'cas...",6,"['oven@ 350', 'in a large bowl combine first 5...","i think i found this recipe in my ""goody bag"" ...","['cream of chicken soup', 'sour cream', 'marga...",...,Jun,2004,0,0,0,0,0,94740,5.0,1


Shape of Sampled Dataset: (21226, 28)


In [146]:
# import libraries
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kelly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [147]:
# Create a function for tokenizer

stemmer = nltk.stem.PorterStemmer()
ENGLISH_STOP_WORDS = stopwords.words('english')

def recipe_tokenizer(sentence):
    # remove punctuation and set to lower case
    for punctuation_mark in string.punctuation:
        sentence = sentence.replace(punctuation_mark,'').lower()

    # split sentence into words
    listofwords = sentence.split(' ')
    listofstemmed_words = []

    # remove stopwords and any tokens that are just empty strings
    for word in listofwords:
        if (not word in ENGLISH_STOP_WORDS) and (word!=''):
            # Stem words
            stemmed_word = stemmer.stem(word)
            listofstemmed_words.append(stemmed_word)

    return listofstemmed_words

In [148]:
# Import libraries
import gensim
from gensim.models import Word2Vec
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [176]:
# Function for word embedding using Word2Vec
def word_embedding(sampled_data, column):
    # Tokenize the text data
    tokenized_data = sampled_data[column].apply(recipe_tokenizer)

    # Train a Word2Vec model
    model = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Create word embeddings for each word in the vocabulary
    embeddings = {word: model.wv[word] for word in model.wv.index_to_key}

    # Save the trained Word2Vec model
    model.save('word2vec_model.bin')

    return embeddings


In [177]:
# Function to pre-compute and store the combined embeddings
def precompute_embeddings(sampled_data):
    # Step 1: Process 'ingredients' using word2vec and create word embeddings
    embeddings = word_embedding(sampled_data, 'ingredients')

    # Step 2: Concatenate relevant columns (excluding 'ingredients')
    sampled_data['text_data'] = sampled_data[['name', 'tags', 'description']].astype(str).agg(' '.join, axis=1)

    # Step 3: Preprocess the text data (example: lowercase conversion)
    sampled_data['text_data'] = sampled_data['text_data'].str.lower()

    # Step 4: Vectorize the text data (excluding 'ingredients') using TF-IDF
    vectorizer = TfidfVectorizer(min_df=5,
                                 tokenizer=recipe_tokenizer)
    vectorized_data = vectorizer.fit_transform(sampled_data['text_data'])

    # Step 5: Retrieve the word embeddings for 'ingredients'
    ingredient_embeddings = [np.mean([embeddings[word] for word in recipe_tokenizer(ingredients) if word in embeddings]
                                      or [np.zeros(100)], axis=0) for ingredients in sampled_data['ingredients']]

    # Step 6: Combine the vectorized data and ingredient embeddings
    combined_embeddings = np.concatenate([vectorized_data.toarray(), np.array(ingredient_embeddings)], axis=1)
    
    # Step 7: Store combined embeddings in pkl file
    with open('combined_embeddings.pkl', 'wb') as f:
        pickle.dump(combined_embeddings, f)
    
    # Step 8: Store the trained TF-IDF vectorizer model in a separate pkl file
    with open('tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)
    
    # Step 9: Done!  
    print("Text data and TF-IDF vectorizer model stored in pkl files!")
    


In [178]:
# store vectorized data and the trained TF-IDF vectorizer model from sampled data
precompute_embeddings(sampled_data)

Text data and TF-IDF vectorizer model stored in pkl files!


In [227]:
# Function to load the Word2Vec model
def load_word2vec_model():
    return Word2Vec.load('word2vec_model.bin')

# Function to load the combined embeddings and TF-IDF vectorizer model
def load_embeddings_and_vectorizer():
    with open('combined_embeddings.pkl', 'rb') as f:
        combined_embeddings = pickle.load(f)
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
    return combined_embeddings, vectorizer

# Function for finding recipes
def find_similar_recipes(sampled_data, user_input, num_similar=5):
    # Load the Word2Vec model
    word2vec_model = load_word2vec_model()

    try:
        combined_embeddings, vectorizer = load_embeddings_and_vectorizer()
    except FileNotFoundError:
        precompute_embeddings(sampled_data)
        combined_embeddings, vectorizer = load_embeddings_and_vectorizer()

    # Process user input
    # Create a DataFrame for user input
    user_data = pd.DataFrame({'text_data': [user_input]})
    user_data['text_data'] = user_data['text_data'].str.lower()

    # Tokenize and create word embeddings for user input
    user_tokens = recipe_tokenizer(user_data['text_data'][0])
    user_doc_embeddings = []
    for token in user_tokens:
        try:
            embedding = word2vec_model.wv.get_vector(token)
            user_doc_embeddings.append(embedding)
        except KeyError:
            # Token not found in Word2Vec model, ignore it
            pass

    if not user_doc_embeddings:
        # If none of the user's input tokens are in the Word2Vec model, create a zero vector
        user_doc_embedding = np.zeros(word2vec_model.vector_size)
    else:
        user_doc_embedding = np.mean(user_doc_embeddings, axis=0)

    # Ensure the number of features in user_vectorized_data matches with combined_embeddings
    num_missing_features = combined_embeddings.shape[1] - user_doc_embedding.shape[0]
    if num_missing_features > 0:
        # Add zero columns to user_doc_embedding to match the number of features
        user_doc_embedding = np.pad(user_doc_embedding, (0, num_missing_features))

    # Combine the vectorized data and user input embeddings
    user_vectorized_data = vectorizer.transform(user_data['text_data']).toarray()
    vector_size_match = min(user_vectorized_data.shape[1], word2vec_model.vector_size)
    user_doc_embedding = user_doc_embedding[:vector_size_match]
    combined_user_embeddings = np.concatenate([user_vectorized_data, user_doc_embedding.reshape(1, -1)], axis=1)

    # Slice the matrix to keep only the relevant features
    combined_user_embeddings = combined_user_embeddings[:, :combined_embeddings.shape[1]]

    # Compute cosine similarity with user input
    cosine_sim_matrix = cosine_similarity(combined_user_embeddings, combined_embeddings)

    # Retrieve similar recipe indices
    similar_recipes = cosine_sim_matrix[0].argsort()[::-1][:num_similar]

    # Get similar recipe names from sampled_data
    similar_recipe_names = sampled_data.iloc[similar_recipes]['name'].tolist()

    return similar_recipe_names



In [233]:
# Test
find_similar_recipes(sampled_data, "japanese noodles")

['acadia s baked eggplant',
 'janae s spaghetti',
 'cheesy rice  mushrooms and peas',
 'slow cooker spaghetti   meatballs',
 'ramen with peas and parmesan']

In [ ]:
# Function for finding recipes
def find_similar_recipes(food_df, recipe_index, num_similar=5):
    # Step 1: Process 'ingredients' using word2vec and create word embeddings
    embeddings = word_embedding(food_df, 'ingredients')

    # Step 2: Concatenate relevant columns (excluding 'ingredients')
    food_df['text_data'] = food_df[['name', 'tags', 'description']].astype(str).agg(' '.join, axis=1)

    # Step 3: Preprocess the text data (example: lowercase conversion)
    food_df['text_data'] = food_df['text_data'].str.lower()

    # Step 4: Vectorize the text data (excluding 'ingredients') using TF-IDF
    vectorizer = TfidfVectorizer(min_df=5,
                                 tokenizer=recipe_tokenizer)
    vectorized_data = vectorizer.fit_transform(food_df['text_data'])

    # Step 5: Retrieve the word embeddings for 'ingredients'
    ingredient_embeddings = [np.mean([embeddings[word] for word in recipe_tokenizer(ingredients) if word in embeddings]
                                      or [np.zeros(100)], axis=0) for ingredients in food_df['ingredients']]

    # Step 6: Combine the vectorized data and ingredient embeddings
    combined_embeddings = np.concatenate([vectorized_data.toarray(), np.array(ingredient_embeddings)], axis=1)

    # Step 7: Compute cosine similarity
    cosine_sim_matrix = cosine_similarity(combined_embeddings)

    # Step 8: Retrieve similar recipes
    similar_recipes = cosine_sim_matrix[recipe_index].argsort()[::-1][1:num_similar + 1]  # Exclude the recipe itself

    # Get similar recipe names
    similar_recipe_names = food_df.loc[similar_recipes, 'name'].tolist()

    return similar_recipe_names


In [ ]:
# Modify the batch_processing function to print the name of the recipe being compared
def batch_processing(data_df, batch_size):
    num_rows = data_df.shape[0]
    num_batches = (num_rows // batch_size) + 1

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_rows)

        batch_data = data_df.iloc[start_idx:end_idx]

        # Perform your desired operations on the batch_data
        # For example, you can call your find_similar_recipes function here
        for recipe_index in range(batch_data.shape[0]):
            recipe_name = batch_data.iloc[recipe_index]['name']
            similar_recipes = find_similar_recipes(batch_data, recipe_index, num_similar=5)
            print(f"Batch {i+1}, Recipe Name: {recipe_name}, Similar Recipes:")
            for j, recipe in enumerate(similar_recipes):
                print(f"{j+1}. {recipe}")


In [ ]:
# Test

# Define the batch size
batch_size = 5000

# Call the batch processing function
batch_processing(food_df, batch_size)

Batch 1, Recipe Name: arriba   baked winter squash mexican style, Similar Recipes:
1. granny squash   creamed squash
2. acorn squash roasted with applesauce
3. acorn squash stuffed with lamb   curry
4. acorn squash soup
5. acorn or butternut squash risotto
Batch 1, Recipe Name: a bit different  breakfast pizza, Similar Recipes:
1. a different  pizza
2. alfredo deep dish pizza
3. an aussie vegemite pizza
4. alabama egg and sausage souffle
5. amazing quiche


KeyboardInterrupt: 